In [ ]:
from risk_data import get_factor_data
factor_data = get_factor_data()

In [ ]:
s = factor_data.ret.sel(factor_name='SPY').to_series().dropna()
vol, param = calibrate_garch(s, scale=True)a


# Attempt 2

In [1]:
import pandas as pd
import numpy as np
from numpy import log10
from arch import arch_model


In [2]:
def scale_for_garch(series: pd.Series, target_magnitude = 2) -> pd.Series:
    """
    Scale the input series to a suitable range for GARCH estimation by dividing 
    by an appropriate power of 10.

    Parameters
    ----------
    series : pd.Series
        The input time series to scale.

    Returns
    -------
    pd.Series
        The scaled series.
    """
    power = int(log10(s.abs().max()))
    return s * 10**(-(power - target_magnitude))


def calibrate_garch(series: pd.Series, scale=False, **kwargs):
    """
    Calibrate a GARCH model on the given time series and return the forecasted 
    volatility for the next period, along with the model parameters.

    Parameters
    ----------
    series : pd.Series
        The return series to fit the GARCH model to.
    **kwargs
        Additional keyword arguments passed to the GARCH model, including `p` and `q`.

    Returns
    -------
    vol_forecast : float
        The forecasted volatility for the next period.
    params : pd.Series
        The estimated parameters of the fitted GARCH model.
    """
    series_scaled = scale_for_garch(series) if scale else series
    
    model = arch_model(series_scaled, vol='Garch', **kwargs)
    fit = model.fit(disp="off")
    vol_forecast = fit.conditional_volatility[-1]
    params = fit.params
    return vol_forecast, params


def expanding_garch_forecast(series: pd.Series, start_window=250, **garch_kwargs):
    """
    Apply an expanding-window GARCH model to a time series, iteratively fitting 
    the model and forecasting volatility for each time step.

    Parameters
    ----------
    series : pd.Series
        The return series, indexed by date, to which the GARCH model is applied.
    start_window : int, optional, default 250
        The minimum number of observations to start the first GARCH fit.
    **garch_kwargs
        Additional keyword arguments for the GARCH model, including `p` and `q`.

    Returns
    -------
    pd.Series
        Forecasted next-day volatilities.
    pd.DataFrame
        The estimated parameters at each step.
    """
    series = series.dropna()
    
    results = {
        series.index[i]: calibrate_garch(series.iloc[:i+1], **garch_kwargs)
        for i in range(start_window, len(series))
    }
    
    forecasts = pd.Series({date: vol for date, (vol, _) in results.items()})
    param_df = pd.DataFrame({date: params for date, (_, params) in results.items()}).T
    
    return forecasts, param_df


In [ ]:
def calibrate_garch(series: pd.Series, scale=False, **kwargs):
    """
    Calibrate a GARCH model on the given time series and return the forecasted 
    volatility for the next period, along with the model parameters.

    Parameters
    ----------
    series : pd.Series
        The return series to fit the GARCH model to.
    **kwargs
        Additional keyword arguments passed to the GARCH model, including `p` and `q`.

    Returns
    -------
    vol_forecast : float
        The forecasted volatility for the next period.
    params : pd.Series
        The estimated parameters of the fitted GARCH model.
    """
    series_scaled = scale_for_garch(series) if scale else series
    
    model = arch_model(series_scaled, vol='GARCH', **kwargs)
    fit = model.fit(disp="off")
    vol_forecast = fit.conditional_volatility[-1]
    params = fit.params
    return vol_forecast, params


In [ ]:
s = factor_data.ret.sel(factor_name='SPY').to_series().dropna()
calibrate_garch(s, scale=False)

In [ ]:
# # Example usage: Create a sample time series (random data for illustration)
# np.random.seed(0)  # For reproducibility
# data = pd.Series(np.random.randn(1000) * 1000, 
#                  index=pd.date_range(start="2020-01-01", periods=1000, freq="D"))

# # Apply the expanding GARCH model with p=1, q=1
# forecast_vols, param_df = expanding_garch_forecast(data, start_window=250, p=1, q=1)

# # Check the results
# forecast_vols.head(), param_df.head()
